# Implement an informational measure of convexity 

This is based on Shane's implementation of the measure of monotonicity

In [41]:
import numpy as np
from various_copied import generate_list_models, measure_monotonicity, upward_monotonicity_entropy

In [2]:
def binary_to_int(arr):
    """ Converts a 2-D numpy array of 1s and 0s into integers, assuming each
    row is a binary number.  By convention, left-most column is 1, then 2, and
    so on, up until 2^(arr.shape[1]).

    :param arr: 2D numpy array
    :returns: 1D numpy array, length arr.shape[0], containing integers
    """
    return arr.dot(1 << np.arange(arr.shape[-1]))

def get_preds(num_arr, num):
    """
    Given an array of ints, and an int, get all predecessors of the
    model corresponding to int.
    Returns an array of same shape as num_arr, but with bools
    """
    return num_arr & num == num_arr

def get_succes(num_arr, num):
    return num_arr & num == num

def has_true_pred(num_arr, quantifier, num):
    """
    checks if num has at least one true predecessor in num_array for the quantifier
    """
    preds = get_preds(num_arr, num)
    return np.any(quantifier * preds)

def has_true_contour(num_arr, quantifier, num):
    """
    check if num has at least one true contour (true predecessor and true successor)
    """
    preds = get_preds(num_arr, num)
    # find successors
    succes = get_succes(num_arr, num)
    return np.any(quantifier * preds) and np.any(quantifier * succes)

Check that the get_succes function works:

In [3]:
all_models = generate_list_models(3)

In [4]:
all_models

array([[0, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 1]])

In [5]:
get_preds(
    binary_to_int(all_models), 
    binary_to_int(np.array([1, 1, 0]))
).reshape(-1,1)

array([[ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False]])

Basically the only thing I need to modify from Shane's version is that instead of conditionalizing on whether the model has a true submodel, I conditionalize on whether the model has both a true submodel and a true supermodel.

In [72]:
def convexity_entropy(all_models, quantifier):
    """
    Measures degree of convexity of a quantifiers as
    1 - H(Q | true_contour) / H(Q) where 
    (1) H is (conditional) entropy, and 
    (2) true_contour is whether there's a true submodel and a true supermodel
    """

    quantifier = quantifier.flatten()

    if np.all(quantifier) or not np.any(quantifier):
        return 1
    
    # calculate entropy of 1_Q. Average surprisal of quantifier being true.
    p_q_true = sum(quantifier) / len(quantifier)
    p_q_false = 1 - p_q_true
    q_ent = - (p_q_true*np.log2(p_q_true) + p_q_false*np.log2(p_q_false))

    # get integers corresponding to each model
    model_ints = binary_to_int(all_models)

    # vector of length quantifier, 
    # 1 if that model has a true contour, 0 otherwise
    true_contour = np.vectorize(
        lambda num: has_true_contour(model_ints, quantifier, num)
    )(model_ints).astype(int)

    # probability that a random model has a true predecessor (p_pred) and that it doesn't (p_nopred)
    p_contour = sum(true_contour) / len(true_contour)
    p_nocontour = 1 - p_contour

    # prob that quant is true & model has a true pred
    q_contour = sum(quantifier * true_contour) / len(quantifier)
    # prob that quant is true & model has no true pred
    q_nocontour = sum(quantifier * (1 - true_contour)) / len(quantifier)
    # prob that quant is false & model has a true pred
    noq_contour = sum((1 - quantifier) * true_contour) / len(quantifier)
    # prob that quant is false & model has no true pred
    noq_nocontour = sum((1 - quantifier) * (1 - true_contour)) / len(quantifier)

    # 
    contour_logs = np.log2([noq_contour, q_contour] / p_contour)
    contour_logs[contour_logs == -np.inf] = 0
    nocontour_logs = np.log2([noq_nocontour, q_nocontour] / p_nocontour)
    nocontour_logs[nocontour_logs == -np.inf] = 0
    print("noq_nocontour: ", noq_nocontour, "q_nocontour: ", q_nocontour, " p_nocontour: ", p_nocontour)
    
    ent_contour = -np.nansum(np.array([noq_contour, q_contour]) * contour_logs)
    ent_nocontour = -np.nansum(np.array([noq_nocontour, q_nocontour]) * nocontour_logs)
    print("Entropy contour: ", ent_contour, " Ent no contour: ", ent_nocontour)
    cond_ent = ent_contour + ent_nocontour
    print("True cont: ", true_contour)
    # return 0 if q_ent == 0 else 1 - (cond_ent / q_ent)
    return 1 - cond_ent / q_ent


def measure_convexity(all_models, quantifier, measure=convexity_entropy):
    interpretations = [
        measure(all_models, quantifier),
        measure(all_models, 1 - quantifier),
        # downward monotonicity
        measure(1 - all_models, 1 - quantifier),
        measure(1 - all_models, quantifier)]
    return np.max(interpretations)

In [13]:
all_models = generate_list_models(4)

In [105]:
random_quant = np.random.randint(0,2,size=(len(all_models),1))
# random_quant = ((all_models.sum(axis=1)==1)|(all_models.sum(axis=1)==4)).astype(int)

model_ints = binary_to_int(all_models)
true_contour = np.vectorize(
        lambda num: has_true_contour(model_ints, random_quant.flatten(), num)
)(model_ints).astype(int)

for idx, model in enumerate(all_models):
    print(model, "Q:", random_quant[idx], "cont:", true_contour[idx])
    
print("Monotonicity: ", convexity_entropy(all_models, random_quant))
print("Convexity:    ", upward_monotonicity_entropy(all_models, random_quant))

[0 0 0 0] Q: [0] cont: 0
[1 0 0 0] Q: [1] cont: 1
[0 1 0 0] Q: [0] cont: 0
[0 0 1 0] Q: [0] cont: 0
[0 0 0 1] Q: [0] cont: 0
[1 1 0 0] Q: [1] cont: 1
[1 0 1 0] Q: [0] cont: 1
[1 0 0 1] Q: [0] cont: 1
[0 1 1 0] Q: [1] cont: 1
[0 1 0 1] Q: [1] cont: 1
[0 0 1 1] Q: [1] cont: 1
[1 1 1 0] Q: [1] cont: 1
[1 1 0 1] Q: [0] cont: 0
[1 0 1 1] Q: [1] cont: 1
[0 1 1 1] Q: [1] cont: 1
[1 1 1 1] Q: [0] cont: 0
noq_nocontour:  0.375 q_nocontour:  0.0  p_nocontour:  0.375
Entropy contour:  0.4512050593046014  Ent no contour:  -0.0
True cont:  [0 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0]
Monotonicity:  0.5487949406953986
True preds:  [0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
Convexity:     0.3112781244591327


C:\Users\s1569804\AppData\Local\Continuum\anaconda3\envs\deep_learning\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log2


Most of the times, for random quantifiers monotonicity and convexity take the same values.
- When the smallest and largest models get different values, monotonicity and convexity are identical. 
    - Monotonicity and convexity are identical when the models that have a true predecessor are the same models that have a true contour, since the only other component (the quantifier's truth itself) is the same.
- When they are different, convexity is greater than monotonicity (when considering all possible changes of 1s to 0s).

In [119]:
differences = []
for i in range(500):
    quant = np.random.randint(0,2,size=(len(all_models),1))
    differences.append(measure_convexity(all_models, quant) - measure_monotonicity(all_models, quant))

C:\Users\s1569804\AppData\Local\Continuum\anaconda3\envs\deep_learning\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log2
C:\Users\s1569804\AppData\Local\Continuum\anaconda3\envs\deep_learning\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in true_divide
C:\Users\s1569804\AppData\Local\Continuum\anaconda3\envs\deep_learning\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in log2
C:\Users\s1569804\AppData\Local\Continuum\anaconda3\envs\deep_learning\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


In [116]:
differences = np.array(differences)

In [118]:
differences

array([0.        , 0.00632243, 0.00895973, 0.00603691, 0.        ,
       0.        , 0.        , 0.        , 0.01545698, 0.        ,
       0.        , 0.00448651, 0.        , 0.00597827, 0.00866154,
       0.        , 0.0083927 , 0.        , 0.        , 0.        ,
       0.00588474, 0.00794847, 0.        , 0.        , 0.        ,
       0.01164554, 0.        , 0.        , 0.00722388, 0.01814118,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01050007, 0.        , 0.        , 0.        , 0.00653331,
       0.        , 0.01809261, 0.        , 0.        , 0.007514  ,
       0.        , 0.01036809, 0.0093268 , 0.01257436, 0.01443849,
       0.01301536, 0.        , 0.        , 0.        , 0.00314971,
       0.0037048 , 0.        , 0.        , 0.00305143, 0.        ,
       0.01082841, 0.01350531, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00491597, 0.        ,
       0.02131371, 0.        , 0.00518886, 0.        , 0.01203

In [117]:
np.sum(differences!=0)/len(differences)

0.42